In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

class DeliveryEnvironment:
    def __init__(self, width, height, obstacles, deliveries):
        """
        Inicializa o ambiente de entrega, definindo suas dimensões, obstáculos e pontos de entrega.
        
        :param width: Inteiro representando a largura da grade do ambiente.
        :param height: Inteiro representando a altura da grade do ambiente.
        :param obstacles: Lista de tuplas, onde cada tupla contém as coordenadas (x, y) de um obstáculo na grade.
        :param deliveries: Lista de tuplas, onde cada tupla contém as coordenadas (x, y) de um ponto de entrega na grade.
        """
        self.width = width
        self.height = height
        self.obstacles = obstacles
        self.deliveries = deliveries
        self.grid = self.initialize_grid()  # Cria a representação da grade com base nos obstáculos e entregas
        self.agent_position = (4, 4)  # Posição inicial do agente no canto superior esquerdo da grade.

    def initialize_grid(self):
        """
        Cria e retorna uma matriz representando a grade do ambiente, marcando obstáculos e pontos de entrega.
        
        :return: Uma matriz 2D onde cada célula pode ser 0 (espaço livre), -1 (obstáculo) ou 1 (ponto de entrega).
        """
        # Inicializa uma grade 2D com zeros (espaços livres).
        grid = [[0 for _ in range(self.width)] for _ in range(self.height)]
        # Marca os obstáculos na grade com -1.
        for obstacle in self.obstacles:
            grid[obstacle[1]][obstacle[0]] = -1
        # Marca os pontos de entrega na grade com 1.
        for delivery in self.deliveries:
            grid[delivery[1]][delivery[0]] = 1
        return grid

    def move_agent(self, action):
        """
        Move o agente na grade com base na ação escolhida, se possível.
        
        :param action: Inteiro representando a ação a ser tomada pelo agente (0: cima, 1: baixo, 2: esquerda, 3: direita).
        :return: A recompensa associada ao novo estado do agente após a movimentação ou uma penalidade se o movimento for inválido.
        """
        # Mapeia a ação escolhida para um deslocamento na grade.
        moves = {0: (-1, 0), 1: (1, 0), 2: (0, -1), 3: (0, 1)}
        move = moves[action]
        # Calcula a nova posição do agente com base na ação.
        new_position = (self.agent_position[0] + move[0], self.agent_position[1] + move[1])

        # Verifica se a nova posição é válida (dentro dos limites da grade e não é um obstáculo).
        if 0 <= new_position[0] < self.height and 0 <= new_position[1] < self.width and self.grid[new_position[0]][new_position[1]] != -1:
            self.agent_position = new_position  # Atualiza a posição do agente.
            return self.grid[new_position[0]][new_position[1]]  # Retorna a recompensa baseada no estado alcançado.
        else:
            return -1  # Retorna uma penalidade se a ação levar a uma posição inválida.

    def is_delivery_completed(self):
        """
        Verifica se o agente alcançou um ponto de entrega.
        
        :return: Booleano indicando se o agente está em um ponto de entrega.
        """
        # Retorna True se a posição atual do agente é um ponto de entrega, caso contrário False.
        return self.grid[self.agent_position[0]][self.agent_position[1]] == 1
    
    def reset(self):
        """
        Reinicia o ambiente e a posição do agente para o estado inicial.
        """
        # Define a posição do agente de volta ao ponto de partida.
        self.agent_position = (4, 4)


In [ ]:
class QLearningAgent:
    def __init__(self, environment, learning_rate=0.1, discount_factor=0.95, epsilon=0.1):
        """
        Inicializa o agente de Q-learning.
        
        :param environment: O ambiente no qual o agente opera.
        :param learning_rate: Taxa de aprendizado (alfa), que determina o quanto da nova informação substitui a antiga.
        :param discount_factor: Fator de desconto (gamma), que pondera a importância das recompensas futuras.
        :param epsilon: Parâmetro para a política epsilon-greedy, equilibrando exploração e explotação.
        """
        self.environment = environment
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.q_table = np.zeros((environment.height, environment.width, 4))
        self.path = []  # Inicializa a lista para armazenar o caminho percorrido pelo agente

    def choose_action(self):
        """
        Escolhe uma ação usando a política epsilon-greedy.
        
        Com uma probabilidade epsilon, escolhe uma ação aleatória (exploração).
        Com a probabilidade restante, escolhe a melhor ação conhecida (explotação).
        """
        if np.random.rand() < self.epsilon:
            # Exploração: escolhe uma ação aleatoriamente.
            return np.random.randint(4)
        else:
            # Explotação: escolhe a melhor ação conhecida baseada na Q-Table.
            state = self.environment.agent_position
            return np.argmax(self.q_table[state[0], state[1]])

    def update_q_table(self, reward, action, current_state, next_state):
        """
        Atualiza a Q-Table usando a equação de Bellman.
        
        :param reward: A recompensa recebida após tomar a ação.
        :param action: A ação que foi tomada.
        :param current_state: O estado do agente antes da ação ser tomada.
        :param next_state: O estado do agente após a ação ser tomada.
        """
        # Calcula o melhor valor Q futuro para o próximo estado
        future_rewards = np.max(self.q_table[next_state[0], next_state[1]])
        
        # Atualiza o valor Q para o estado atual e ação tomada
        self.q_table[current_state[0], current_state[1], action] += self.learning_rate * (
            reward + self.discount_factor * future_rewards - self.q_table[current_state[0], current_state[1], action]
        )


    def train(self, episodes=1000):
        for episode in range(episodes):
            self.environment.reset()  # Reinicia o ambiente ao estado inicial
            done = False
            # self.path = []  # Limpa o caminho no início de cada episódio

            while not done:
                current_position = self.environment.agent_position  # Posição atual antes da ação
                action = self.choose_action()  # Escolhe a ação
                reward = self.environment.move_agent(action)  # Executa a ação
                next_position = self.environment.agent_position  # Posição após a ação

                # Verifica se a entrega foi completada antes de aplicar a penalidade por viver
                if not self.environment.is_delivery_completed():
                    reward -= 0.1  # Penalidade por viver aplicada somente se a entrega não for completada

                # Atualiza a Q-Table, passando os estados atual e futuro corretamente
                self.update_q_table(reward, action, current_position, next_position)

                print(f"Época {episode + 1}:")
                print(f"Estado anterior: {current_position}")
                print(f"Ação tomada pelo agente: {action}")
                print(f"Estado atual: {next_position}")
                print(f"Recompensa: {reward}\n")

                # Verifica se a entrega foi completada para encerrar o loop
                done = self.environment.is_delivery_completed()

                # Atualiza o caminho
                self.path.append(current_position)  # Adiciona a posição atual ao caminho

In [ ]:
# Exemplo de criação do ambiente e do agente
environment = DeliveryEnvironment(9, 9, [(1, 6), (2, 6), (2, 7), (6, 1), (6, 2), (7, 2),(5,3)], [(7, 1), (1, 7)])
agent = QLearningAgent(environment, learning_rate=0.1, epsilon=0.1, discount_factor=0.9)

In [ ]:
agent.train(episodes=1000)

In [ ]:
len(agent.path)

In [ ]:
# Importações necessárias
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import time
from IPython.display import display, clear_output

# Preparando a representação gráfica do labirinto
fig, ax = plt.subplots()

# Configurações de plotagem
ax.set_xticks([])
ax.set_yticks([])

# Função para plotar o estado do ambiente em um determinado frame
def plot_frame(frame):
    fig, ax = plt.subplots()
    ax.matshow(environment.grid, cmap=plt.cm.Pastel1)
    if frame < len(agent.path):
        state = agent.path[frame]
        ax.plot(state[1], state[0], 'bo', markersize=15)  # Desenha o agente
        ax.set_title(f"Passo {frame + 1}")
    # plt.xticks([]), plt.yticks([])  # Remove os ticks do eixo
    return fig

# Exibir as imagens em sequência para simular a animação
for frame in range(len(agent.path)):
    fig = plot_frame(frame)
    display(fig)  # Mostra a figura
    clear_output(wait=True)  # Limpa a saída para a próxima imagem
    plt.close(fig)  # Fecha a figura para liberar memória
    time.sleep(0.1)  # Velocidade da "animação"

In [ ]:
agent.q_table